In [1]:
!pip install googlesearch-python lxml deepspeed sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#
# Cell 2: Imports
#
# Description: All required libraries for the application are imported here.
#

import os
import json
import socket
from typing import List, Dict
import subprocess
import time
import re
import gradio as gr

# LangChain and related libraries
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from langchain_ollama.chat_models import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings

# Search library
try:
    from googlesearch import search
except ImportError:
    print("Error: 'googlesearch-python' is not installed. Please run 'pip install googlesearch-python'")

# Set a user agent to avoid being blocked by Google search
os.environ["USER_AGENT"] = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0"

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
#
# Cell 3: Agent 1 - User Profiler
#
# Description: This agent is responsible for understanding the user's knowledge level.
# It dynamically generates a questionnaire, analyzes the answers, and creates a
# profile that will be used by Agent 3 to tailor its responses.
#

class Questionnaire(BaseModel):
    questions: List[str] = Field(description="A list of 4-5 questions for the user.")

class UserProfilerAgent_V3:
    """
    Agent 1 (V3): Guarantees the user's name is collected first before
    using an LLM to dynamically generate the rest of the questionnaire.
    **ADAPTED FOR MODULAR, GRADIO-FRIENDLY USE.**
    """
    def __init__(self, profiles_dir: str = "user_profiles"):
        self.profiles_dir = profiles_dir
        if not os.path.exists(self.profiles_dir):
            os.makedirs(self.profiles_dir)

        try:
            host_node = socket.gethostname()
            # NOTE: The ASURITE ID should be that of the user running the Ollama server.
            # This is specified as a hackathon resource.
            asurite_id = "kvinod"
            self.llm = ChatOllama(model="qwen3:14b", base_url=f"http://{asurite_id}@{host_node}:11434/")
            self.structured_llm = self.llm.with_structured_output(Questionnaire)
            print("✅ [Agent 1] Successfully connected to Ollama LLM.")
        except Exception as e:
            print(f"❌ [Agent 1] Error connecting to Ollama: {e}")
            self.llm = None

    def get_user_list(self) -> List[str]:
        """Scans the profiles directory and returns a list of user names."""
        if not os.path.exists(self.profiles_dir):
            return []
        files = [f for f in os.listdir(self.profiles_dir) if f.endswith('.txt')]
        # Convert 'first_last.txt' to 'First Last'
        names = [" ".join(f.replace('.txt', '').split('_')).title() for f in files]
        return names

    def _generate_questions_with_llm(self) -> List[str]:
        """Uses an LLM to dynamically generate a user questionnaire."""
        if not self.llm: # Fallback if LLM is not available
            return [
                "On a scale of 1-5, how comfortable are you with Python?",
                "Which Python data science libraries (like Pandas or NumPy) have you used before?",
                "Have you ever heard of using GPUs to speed up data analysis?",
                "What's the first tool you'd reach for to do a large matrix multiplication in Python?"
            ]
            
        print("\n🤖 [Agent 1] Generating a personalized questionnaire...")
        prompt = PromptTemplate(
            template="""
            You are a helpful assistant for an AI Data Science Tutor. Your goal is to create a short questionnaire (4-5 questions) to understand a user's knowledge level.
            The questions should gently probe their experience with:
            1. The Python programming language.
            2. Common CPU-based data science libraries (like NumPy, Pandas).
            3. Their awareness of GPU computing and hardware acceleration.
            4. Their familiarity with any NVIDIA-specific GPU libraries (like CuPy or RAPIDS).
            IMPORTANT: Do NOT ask for the user's name, as it will be collected separately.
            Return the questions as a JSON list. Be conversational and friendly.
            """,
            input_variables=[],
        )
        query_generation_chain = prompt | self.structured_llm
        try:
            response_model = query_generation_chain.invoke({})
            return response_model.questions
        except Exception as e:
            print(f"-> [Agent 1] LLM failed to generate questions, falling back to default. Error: {e}")
            return [
                "On a scale of 1-5, how comfortable are you with Python?",
                "Which Python data science libraries (like Pandas or NumPy) have you used before?",
                "Have you ever heard of using GPUs to speed up data analysis?",
                "What's the first tool you'd reach for to do a large matrix multiplication in Python?"
            ]

    def generate_and_save_report(self, user_name: str, answers_dict: dict) -> str:
        """
        Takes a user name and a dictionary of answers, generates a report with an LLM,
        and saves it to a file. Returns the path to the saved file.
        """
        if not self.llm:
            return "Error: LLM not connected."

        print(f"\n🤖 [Agent 1] Analyzing responses for {user_name} and creating a profile...")
        answers_str = "\n".join([f"- {q}: {a}" for q, a in answers_dict.items()])
        prompt = PromptTemplate(
            template="""
            You are an expert AI analyst. A user named {user_name} has answered a questionnaire about their data science skills.
            Your task is to analyze their answers and generate a "TUTORING STRATEGY" report for our AI Tutor.
            **User's Answers:**
            {answers}
            **Your Task:**
            1.  Determine the user's knowledge level: 'Beginner', 'Intermediate', or 'Advanced'.
            2.  Write a concise report following the correct strategy format below. This report will be given to another AI, so the instructions must be clear.
            ---
            **STRATEGY FORMATS (Choose ONE):**
            **If 'Beginner':**
            Start with `Knowledge Level: Beginner`. On the next line, start with `TUTORING STRATEGY: The user is a beginner.` Then, explain that the tutor should use high-level concepts, explain the 'why' of GPU acceleration, and introduce NVIDIA libraries (like CuPy) as a simple, powerful alternative.
            **If 'Intermediate':**
            Start with `Knowledge Level: Intermediate`. On the next line, start with `TUTORING STRATEGY: The user is at an intermediate level.` Then, explain that the tutor should provide direct code comparisons (e.g., NumPy vs. CuPy), focus on performance benefits, and show clear benchmarking examples.
            **If 'Advanced':**
            Start with `Knowledge Level: Advanced`. On the next line, start with `TUTORING STRATEGY: The user is advanced.` Then, explain that the tutor can provide nuanced advice, discuss the broader NVIDIA RAPIDS ecosystem, and cover specific benchmarking methodologies on the Sol supercomputer.
            ---
            Now, generate the complete report.
            """,
            input_variables=["user_name", "answers"],
        )
        report_generation_chain = prompt | self.llm
        response_message = report_generation_chain.invoke({"user_name": user_name, "answers": answers_str})
        report_content = response_message.content
        full_report = f"--- User Profile for {user_name} ---\n{report_content}\n--- End of Profile ---"

        # Save the report to a text file
        filename = "_".join(user_name.lower().split()) + ".txt"
        filepath = os.path.join(self.profiles_dir, filename)
        with open(filepath, "w") as f:
            f.write(full_report)
        print(f"\n✅ [Agent 1] User profile report saved successfully to: {filepath}")
        return filepath

In [4]:
#
# Cell 4: Agent 4 - Sol Benchmarker
#
# Description: This agent is responsible for executing code on the Sol supercomputer
# to benchmark the performance difference between CPU and GPU implementations. It writes
# and submits a SLURM batch script.
#

class SolBenchmarker:
    def __init__(self, user: str, python_env: str = "rapids-25.02"): # Note: Kernel name from docs
        if not user or user == "YOUR_ASURITE_ID":
            raise ValueError("A valid ASURITE username is required for SolBenchmarker.")
        self.user = user
        self.python_env = python_env

    def _generate_sbatch_script(self, script_dir: str, cpu_script_name: str, gpu_script_name: str) -> str:
        # This SLURM script is configured according to the hackathon's resources 
        return f"""#!/bin/bash
#SBATCH -p general
#SBATCH -q public
#SBATCH -G 1
#SBATCH --reservation=hackathon2025
#SBATCH -t 0-00:10:00
#SBATCH -c 1
#SBATCH -o {script_dir}/slurm-%j.out
#SBATCH -e {script_dir}/slurm-%j.err

module load mamba/latest
source activate {self.python_env}

echo "--- STARTING CPU BENCHMARK ---"
/usr/bin/time -p python3 {script_dir}/{cpu_script_name} 2>&1
echo "--- FINISHED CPU BENCHMARK ---"

echo ""
echo "--- STARTING GPU BENCHMARK ---"
/usr/bin/time -p python3 {script_dir}/{gpu_script_name} 2>&1
echo "--- FINISHED GPU BENCHMARK ---"
"""
    def _parse_output(self, output_content: str) -> dict:
        try:
            # --- FIX 1: Removed erroneous backslash before the string literal ---
            # This regex finds the execution time from the '/usr/bin/time' command output.
            real_times = re.findall(r"real\s+([\d.]+)", output_content)
            
            cpu_time = float(real_times[0]) if len(real_times) > 0 else None
            gpu_time = float(real_times[1]) if len(real_times) > 1 else None
            print("cpu_time_seconds", cpu_time, "gpu_time_seconds", gpu_time)
            return {"status": "success", "cpu_time_seconds": cpu_time, "gpu_time_seconds": gpu_time}
        except (IndexError, ValueError) as e:
            return {"status": "error", "message": f"Failed to parse benchmark times. Error: {e}", "raw_log": output_content}

    def run_benchmark(self, cpu_code: str, gpu_code: str) -> dict:
        benchmark_dir = os.path.join(os.getcwd(), "benchmark_files")
        os.makedirs(benchmark_dir, exist_ok=True)
        
        cpu_script_path = os.path.join(benchmark_dir, "cpu_benchmark.py")
        gpu_script_path = os.path.join(benchmark_dir, "gpu_benchmark.py")
        sbatch_path = os.path.join(benchmark_dir, "benchmark_job.sh")

        try:
            with open(cpu_script_path, "w") as f: f.write(cpu_code)
            with open(gpu_script_path, "w") as f: f.write(gpu_code)
            sbatch_script = self._generate_sbatch_script(benchmark_dir, "cpu_benchmark.py", "gpu_benchmark.py")
            with open(sbatch_path, "w") as f: f.write(sbatch_script)

            process = subprocess.run(f"sbatch {sbatch_path}", shell=True, capture_output=True, text=True)
            if process.returncode != 0: raise RuntimeError(f"sbatch submission failed: {process.stderr}")

            # --- FIX 2: Removed erroneous backslash before the string literal ---
            # This regex finds the Job ID from the sbatch submission output.
            job_id_match = re.search(r"Submitted batch job (\d+)", process.stdout.strip())

            if not job_id_match: raise RuntimeError(f"Could not parse Job ID from sbatch output: {process.stdout}")
            job_id = job_id_match.group(1)
            print(f"--> [Agent 4] Submitted benchmark job to SLURM with ID: {job_id}")

            print("--> [Agent 4] Waiting for job to complete...")
            while True:
                queue_process = subprocess.run(f"squeue -u {self.user} -j {job_id}", shell=True, capture_output=True, text=True)
                if job_id not in queue_process.stdout: break
                time.sleep(10)

            print(f"--> [Agent 4] Job {job_id} completed.")
            output_file_path = os.path.join(benchmark_dir, f"slurm-{job_id}.out")
            
            if not os.path.exists(output_file_path):
                 err_file_path = os.path.join(benchmark_dir, f"slurm-{job_id}.err")
                 if os.path.exists(err_file_path):
                     with open(err_file_path, "r") as f: error_content = f.read()
                     return {"status": "error", "message": f"Job failed. See error log: {error_content}"}
                 return {"status": "error", "message": f"Output file not found."}

            with open(output_file_path, "r") as f: output_content = f.read()
            return self._parse_output(output_content)
        except Exception as e:
            return {"status": "error", "message": str(e)}

In [5]:
#
# Cell 5: Agents 2 & 3 - Dynamic RAG Pipeline
#
# Description: Agent 2 generates search queries and retrieves relevant documents from the web.
# Agent 3 takes this context, along with the user's profile from Agent 1, to generate
# a tailored, GPU-first answer. It then coordinates with Agent 4 for benchmarking.
#

class SearchQueryGenerator(BaseModel):
    queries: List[str] = Field(description="A list of targeted, keyword-focused search queries.")

def generate_search_queries(query: str, llm) -> List[str]:
    print("-> [Agent 2] Using LLM to generate search queries...")
    prompt_template = PromptTemplate(
        template="""
        You are an expert at generating web search queries.
        Analyze the user's question to identify the core technical task and the programming language.
        Generate 5 concise, targeted search queries. Two for the standard, CPU-based library and three for potential NVIDIA GPU-accelerated libraries.
        User Question: \"{question}\"
        Generate a JSON list of 5 search query strings.
        """,
        input_variables=["question"],
    )
    query_generation_chain = prompt_template | llm.with_structured_output(SearchQueryGenerator)
    try:
        response_model = query_generation_chain.invoke({"question": query})
        print(f"-> [Agent 2] Generated queries: {response_model.queries}")
        return response_model.queries
    except Exception as e:
        print(f"-> [Agent 2] LLM failed to generate structured output: {e}")
        return []

def dynamic_search_agentic(queries: List[str]) -> list[str]:
    print("-> [Agent 2] Executing dynamic search...")
    all_urls = set()
    for q in queries:
        try:
            enhanced_query = f"{q} site:developer.nvidia.com OR site:medium.com/rapids-ai OR site:medium.com/cupy-team"
            search_results = list(search(enhanced_query, num_results=2))
            all_urls.update(url for url in search_results if url)
        except Exception as e:
            print(f"An error occurred during search for query '{q}': {e}")
    final_urls = list(all_urls)
    print(f"-> [Agent 2] Found {len(final_urls)} unique URLs.")
    return final_urls

def _extract_python_code(markdown_text: str) -> Dict[str, str]:
    code_pattern = r"```python\n(.*?)```"
    gpu_heading_pattern = r"### Recommended GPU Solution.*?\n"
    cpu_heading_pattern = r"### Standard CPU Solution.*?\n"
    
    gpu_section_match = re.search(gpu_heading_pattern, markdown_text, re.DOTALL | re.IGNORECASE)
    cpu_section_match = re.search(cpu_heading_pattern, markdown_text, re.DOTALL | re.IGNORECASE)
    
    gpu_code = ""
    if gpu_section_match:
        section_after_heading = markdown_text[gpu_section_match.end():]
        code_match = re.search(code_pattern, section_after_heading, re.DOTALL)
        if code_match:
            gpu_code = code_match.group(1).strip()

    cpu_code = ""
    if cpu_section_match:
        section_after_heading = markdown_text[cpu_section_match.end():]
        code_match = re.search(code_pattern, section_after_heading, re.DOTALL)
        if code_match:
            cpu_code = code_match.group(1).strip()
            
    return {"cpu_code": cpu_code, "gpu_code": gpu_code}


def process_with_rag(query: str, user_profile_path: str = None) -> str:
    print("\n--- Running RAG Pipeline ---")
    host_node = socket.gethostname()
    asurite_id = "kvinod" 
    llm = ChatOllama(model="qwen3:14b", base_url=f"http://{asurite_id}@{host_node}:11434/")

    search_queries = generate_search_queries(query, llm)
    urls = dynamic_search_agentic(search_queries) if search_queries else []

    context_text = ""
    if urls:
        print("-> [Agent 2] Loading and processing context from URLs...")
        # Using a persistent directory for ChromaDB
        persist_directory = 'chroma_db_rag'
        #embedding_model = OllamaEmbeddings(model="nomic-embed-text", base_url=f"http://{asurite_id}@{host_node}:11434/")
        embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        docs = [WebBaseLoader(url).load() for url in urls]
        docs_list = [item for sublist in docs for item in sublist]
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=100)
        doc_splits = text_splitter.split_documents(docs_list)
        
        vectorstore = Chroma.from_documents(
            documents=doc_splits, 
            embedding=embedding_model,
            persist_directory=persist_directory
        )
        retriever = vectorstore.as_retriever()
        retrieved_docs = retriever.invoke(query)
        context_text = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])
        vectorstore.delete_collection() # Clean up

    # --- NEW: Read user profile if provided ---
    user_profile_content = "No user profile provided. Proceed with a general approach."
    if user_profile_path and os.path.exists(user_profile_path):
        with open(user_profile_path, 'r') as f:
            user_profile_content = f.read()
            print(f"-> [Agent 3] Successfully loaded user profile: {user_profile_path}")

    final_prompt_template = PromptTemplate(
        template="""
        You are an AI Data Science Tutor. Your primary mission is to educate users on leveraging NVIDIA-based GPU libraries.
        You have been given a strategy document for how to interact with the current user.

        **TUTORING STRATEGY DOCUMENT:**
        {user_profile}

        **USER'S QUESTION:**
        {question}

        **RELEVANT CONTEXT FROM WEB SEARCH (Use if helpful):**
        {context}

        **YOUR TASK:**
        Based on the TUTORING STRATEGY, the user's QUESTION, and any relevant CONTEXT, provide a conversational and helpful answer.
        - If the strategy says the user is a beginner, be encouraging and explain the 'why' before showing code.
        - If intermediate, provide clear code comparisons and focus on performance.
        - If advanced, feel free to discuss the broader ecosystem (like RAPIDS) and nuances.
        - **ALWAYS prioritize NVIDIA GPU solutions.** If one exists, present it first with a `### Recommended GPU Solution (with [Library])` heading and runnable Python code. Then, show the `### Standard CPU Solution (with [Library])` for comparison.
        - Add a "Performance Note" after the GPU code explaining the benefits and trade-offs.
        - If a direct NVIDIA-based library doesn't exist, provide the standard solution and end with the exact note: "Note: The provided solution is the standard method for this task, as a direct NVIDIA-based GPU library for it is not common."

        YOUR FINAL ANSWER:
        """,
        input_variables=["user_profile", "question", "context"],
    )

    final_chain = final_prompt_template | llm
    llm_response_text = final_chain.invoke({
        "user_profile": user_profile_content,
        "question": query,
        "context": context_text
    }).content
    print("--> [Agent 3] Generated conversational answer.")

    extracted_code = _extract_python_code(llm_response_text)
    cpu_code, gpu_code = extracted_code["cpu_code"], extracted_code["gpu_code"]

    if cpu_code and gpu_code:
        print("--> [Agent 3] Both CPU and GPU code found. Invoking Agent 4 for benchmarking.")
        try:
            benchmarker = SolBenchmarker(user=asurite_id) # The user running the benchmark job
            benchmark_results = benchmarker.run_benchmark(cpu_code, gpu_code)
            if benchmark_results.get("status") == "success":
                cpu_time = benchmark_results.get('cpu_time_seconds')
                gpu_time = benchmark_results.get('gpu_time_seconds')
                cpu_time_str = f"{cpu_time:.4f} seconds" if cpu_time is not None else "N/A"
                gpu_time_str = f"{gpu_time:.4f} seconds" if gpu_time is not None else "N/A"
                benchmark_md = f"""
---
### 📊 Real-World Benchmark (ASU's Sol Supercomputer)
| Metric | Result |
|---|---|
| CPU Time | {cpu_time_str} |
| GPU Time | {gpu_time_str} |
"""
                if cpu_time and gpu_time and gpu_time > 0:
                    speedup = cpu_time / gpu_time
                    benchmark_md += f"| **Speedup** | **{speedup:.2f}x faster on GPU!** |\n"
                llm_response_text += benchmark_md
            else:
                llm_response_text += f"\n\n---\n### ⚠️ Benchmark Failed\n{benchmark_results.get('message')}"
        except ValueError as e:
            llm_response_text += f"\n\n---\n### ⚠️ Benchmark Skipped\n{e}"
    else:
        print("--> [Agent 3] Did not find both code types. Skipping benchmark.")
        
    print("--- Pipeline Complete ---")
    return llm_response_text

# Gradio

In [ ]:
#
# Cell 6: Gradio User Interface
#
# Description: This cell builds the complete Gradio app, integrating all agents.
# It includes controls for creating and selecting user profiles, a chat interface for
# posing questions to the tutor, and logic to connect the selected user profile to
# the RAG pipeline.
#

# --- Instantiate Agent 1 for use in the UI ---
profiler_agent = UserProfilerAgent_V3()

with gr.Blocks(theme=gr.themes.Soft(), css=".gradio-container {background-color: #f5f5f5;}") as demo:
    
    # --- State Management for user profiles ---
    user_state = gr.State({
        "all_users": profiler_agent.get_user_list(),
        "current_user": None
    })

    gr.Markdown("# 🤖 AI Accelerated Data Science Tutor")
    
    with gr.Row():
        with gr.Column(scale=4):
            gr.Markdown("Ask a question about a data science task. For a personalized response, create or select a user profile.")
        with gr.Column(scale=2, min_width=300):
            # --- User Management UI ---
            user_dropdown = gr.Dropdown(
                label="Current User",
                choices=user_state.value["all_users"],
                interactive=True
            )
            with gr.Row():
                new_user_btn = gr.Button("✚ New / Update User")

    gr.Markdown("---")

    # --- Main Chat UI ---
    chatbot = gr.Chatbot(label="Conversation", height=500, type="messages")
    
    with gr.Row():
        msg_textbox = gr.Textbox(
            label="Your Question",
            placeholder="e.g., How do I multiply two 10x10 arrays in Python?",
            scale=4,
            lines=2,
            container=False
        )
        submit_btn = gr.Button("Ask Tutor", variant="primary", scale=1, min_width=150)

    # --- Questionnaire Modal (Hidden by default) ---
    with gr.Group(visible=False) as profiler_ui_group:
        with gr.Blocks() as profiler_modal:
            gr.Markdown("### User Profile Questionnaire")
            gr.Markdown("Please answer the following questions to help me tailor my explanations to your knowledge level.")
            profiler_name_input = gr.Textbox(label="First and Last Name")
            
            q_inputs = [gr.Textbox(label=f"Question {i+1}", visible=False) for i in range(5)]

            submit_profile_btn = gr.Button("Submit Profile")
            cancel_profile_btn = gr.Button("Cancel")

    # ==================================
    # GRADIO HANDLER FUNCTIONS
    # ==================================

    # --- Chat Logic ---
    def on_submit_chat(user_message, chat_history, selected_user):
        chat_history = chat_history or []
        chat_history.append({"role": "user", "content": user_message})
        
        user_profile_path = None
        if selected_user:
            filename = "_".join(selected_user.lower().split()) + ".txt"
            user_profile_path = os.path.join(profiler_agent.profiles_dir, filename)
        else:
            chat_history.append({"role": "assistant", "content": "⚠️ **Warning:** No user profile selected. I'll provide a general answer, but for a personalized experience, please create or select a user profile."})
            yield chat_history, ""
            return

        chat_history.append({"role": "assistant", "content": "Thinking..."})
        yield chat_history, ""
        
        response = process_with_rag(user_message, user_profile_path)
        
        chat_history[-1] = {"role": "assistant", "content": response}
        yield chat_history, ""

    # --- User Profile Logic ---
    def start_profiling_flow(current_user):
        """Called when 'New / Update User' is clicked."""
        questions = profiler_agent._generate_questions_with_llm()
        
        updates = [gr.update(visible=True)]
        updates.append(gr.update(value=current_user if current_user else "", visible=True))
        
        for i in range(5):
            if i < len(questions):
                updates.append(gr.update(label=questions[i], value="", visible=True))
            else:
                updates.append(gr.update(visible=False))
        
        return *updates, questions

    # --- FIX 1: Add 'questions' to the function signature to receive it from the state component. ---
    def process_profile_submission(user_name, state, questions, *answers):
        """Called when 'Submit Profile' is clicked."""
        
        if not user_name:
            gr.Warning("User name cannot be empty!")
            return state, gr.update(choices=state["all_users"], value=state["current_user"]), gr.update(visible=True)

        answers_dict = {q: a for q, a in zip(questions, answers) if q and a}
        profiler_agent.generate_and_save_report(user_name, answers_dict)
        
        gr.Info(f"Profile for {user_name} has been saved!")

        updated_users = profiler_agent.get_user_list()
        state["all_users"] = updated_users
        state["current_user"] = user_name
        
        return state, gr.update(choices=updated_users, value=user_name), gr.update(visible=False)

    def cancel_profiling():
        return gr.update(visible=False)

    def update_current_user(selected_user, state):
        state["current_user"] = selected_user
        gr.Info(f"Switched to user: {selected_user}")
        return state

    # ==================================
    # WIRING UP THE UI EVENTS
    # ==================================
    
    question_state = gr.State([])

    submit_btn.click(
        on_submit_chat,
        [msg_textbox, chatbot, user_dropdown],
        [chatbot, msg_textbox]
    )
    msg_textbox.submit(
        on_submit_chat,
        [msg_textbox, chatbot, user_dropdown],
        [chatbot, msg_textbox]
    )

    new_user_btn.click(
        start_profiling_flow,
        inputs=[user_dropdown],
        outputs=[profiler_ui_group, profiler_name_input] + q_inputs + [question_state]
    )
    
    submit_profile_btn.click(
        process_profile_submission,
        # --- FIX 2: Pass the 'question_state' component as an input. ---
        inputs=[profiler_name_input, user_state, question_state, *q_inputs],
        outputs=[user_state, user_dropdown, profiler_ui_group]
    )

    cancel_profile_btn.click(cancel_profiling, outputs=[profiler_ui_group])

    user_dropdown.change(
        update_current_user,
        inputs=[user_dropdown, user_state],
        outputs=[user_state]
    )

# --- Launch the Application ---
demo.queue().launch(share=True, debug=True)

✅ [Agent 1] Successfully connected to Ollama LLM.
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://f49f5e346eb2e608e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



--- Running RAG Pipeline ---
-> [Agent 2] Using LLM to generate search queries...
-> [Agent 2] Generated queries: ['What is an apple in biology?', 'What is the definition of an apple?', 'What is an apple in the context of technology?', 'What is an apple in the context of the Apple company?', 'What is an apple in the context of computing?']
-> [Agent 2] Executing dynamic search...
-> [Agent 2] Found 6 unique URLs.
-> [Agent 2] Loading and processing context from URLs...
[2025-06-25 20:18:29,787] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/packages/envs/genai25.06/compiler_compat/ld: warning: libstdc++.so.6, needed by /packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.5.0-yaosn2wjlhxqbokllnobo2soiuh6gw3n/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/packages/envs/genai25.06/compiler_compat/ld: /packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.5.0-yaosn2wjlhxqbokllnobo2soiuh6gw3n/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/packages/envs/genai25.06/compiler_compat/ld: /packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.5.0-yaosn2wjlhxqbokllnobo2soiuh6gw3n/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/packages/envs/genai25.06/compiler_compat/ld: /packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.5.0-yaosn2wjlhxqbokllnobo2soiuh6gw3n/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/packages/envs/genai25.06/compiler_

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

-> [Agent 3] Successfully loaded user profile: user_profiles/shreela_p.txt
--> [Agent 3] Generated conversational answer.
--> [Agent 3] Did not find both code types. Skipping benchmark.
--- Pipeline Complete ---
